In [213]:
import pandas as pd
import numpy as np
import re
import os

path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-4:] == '.csv']

df = pd.DataFrame()
for file in files_csv:
    df = pd.concat([df, pd.read_csv(file, sep=',', encoding='utf-8', dtype=str)], ignore_index=True)

df['Motivo'] = ''
df['Motivo general'] = ''
df.dropna(subset=['comentarios'], inplace=True)
df.reset_index(drop=False, inplace=True)


In [176]:
motivos = pd.read_excel('MatrizMotivos.xlsx', dtype=str, skiprows=11)
# index por la columna 'MOTIVO'
motivos = motivos.fillna(method='ffill')
motivos = motivos.set_index('MOTIVO')
# rellenar los indices nan con el valor del indice anterior
motivos = motivos[['NOMBRE_MOTIVO']]

motivos_s = np.array(motivos[['NOMBRE_MOTIVO']])
motivos_ss = []

for i in range(len(motivos_s)):
    motivo_s = motivos_s[i][0]
    motivos_ss.append(motivo_s.replace('-', ' '))



In [211]:
df.head()

,folio,comentarios,Motivo,Motivo general
0,C2S/20220201/00019,[01/02/2022 06:38:03] [EXTERNO] BOTON DE AUXIL...,,
1,C5/20220201/01076,[01/02/2022 11:41:08] [STCPATB71] RADIO: REGIS...,,
2,C5/20220201/02479,[01/02/2022 18:33:22] [C51039856] SE HA REGIST...,,
3,C5/20220201/01776,[01/02/2022 15:11:54] [C5GAME93] SE HA REGISTR...,,
4,C5/20220201/01853,[01/02/2022 15:38:35] [C5GAME93] SE HA REGISTR...,,


In [214]:
df['comentarios'][10713:10716]

10713    [2022-02-02 23:51:06]  [ANPR] sensor id: 55209...
10714    [2022-02-02 00:02:28]  [ANPR] sensor id: 53210...
10715    [2022-02-02 00:08:52]  [ANPR] sensor id: 55407...
Name: comentarios, dtype: object

In [181]:
index=0
for i in range(len(df['comentarios'])):
    index+=1
    print(index, '/', len(df['comentarios']), end='\r')
    for j in range(len(motivos_s)):
        if motivos_s[j][0] in df['comentarios'][i] or motivos_ss[j] in df['comentarios'][i]:
            df['Motivo'][i] = str(motivos_s[j][0])
            break
        elif re.search('F\s*A\s*L\s*S\s*(A|O)(.*A\s*L\s*A\s*R\s*M\s*A)?', df['comentarios'][i], re.MULTILINE | re.IGNORECASE) or 'FALLA EN EL SISTEMA' in df['comentarios'][i]:
            df['Motivo'][i] = 'FALSA-ALARMA'
        elif re.search('I\s*N\s*F\s*O\s*R\s*M\s*A\s*T\s*I\s*V\s*O', df['comentarios'][i], re.MULTILINE | re.IGNORECASE):
            df['Motivo'][i] = 'INFORMATIVO'
        elif re.search('F\s*O\s*L\s*I\s*O\s*D\s*U\s*P\s*L\s*I\s*C\s*A\s*D\s*O', df['comentarios'][i], re.MULTILINE | re.IGNORECASE):
            df['Motivo'][i] = 'FOLIO DUPLICADO'
        else:
            df['Motivo'][i] = np.nan

KeyError: 10714

In [132]:
df['Motivo'].value_counts()

FALSA-ALARMA                          1217
SERVICIOS-ENTREVISTA CIUDADANA         392
AGRESION-PERSONA                       305
ACCIDENTE-CHOQUE SIN LESIONADOS        222
DENUNCIA-PERSONA SOSPECHOSA            209
                                      ... 
ACCIDENTE-VEHICULO DESBARRANCADO         1
ACCIDENTE-VEHICULO ATRAPADO-VARADO       1
ROBO-A TAXISTA                           1
ROBO-CABLE                               1
CADAVER-ATRAGANTAMIENTO-ASFIXIA          1
Name: Motivo, Length: 182, dtype: int64

In [135]:
df = df.dropna(subset=['Motivo'])

In [ ]:
df_grouped = df.groupby('Motivo')

In [136]:
#guardar el df en un archivo xlsx
df.to_excel('bitacora etiquetada.xlsx', index=False)

In [79]:
import spacy
from spacy.lang.es.examples import sentences 
# !python -m spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")

comentarios_lema = []
tags = ['PROPN', 'AUX', 'NOUN', 'PUNCT', 'ADP', 'DET', 'SPACE']
i=0
for comentario in df['comentarios'][:1]:
    i+=1
    print(i, "/", len(df), end='\r')
    doc = nlp(comentario)
    comentario_aux = ''
    for token in doc:
        if token.pos_ not in tags:
            comentario_aux += token.text + '\t' + token.pos_ + '\n'
    comentarios_lema.append(comentario_aux)

In [81]:

with open('comentarios_lema.txt', 'w', encoding='utf-8') as f:
    for comentario in comentarios_lema:
        f.write(comentario)
